<h1 align=center><font size = 5>The Battle of Neighborhoods</font></h1>


# 1. The Problem

Compare the (dis)similarities that multicultural cities have from the perspective of clusters of coffee shops.



## Discussion of the Problem

New York City, Toronto and Paris belong to the top ten multicultural cities in the world. From the "center" of each city we will consider 50 coffee shops within a radius of 1km. We will analyze similarities among clusters of coffee shops in each area. 

For example, is it true that in all clusters of coffee shops there are an executive offices, or supermarkets, or restaurants, or parks nearby?

This problem fits to the audience that wish to open a coffee shop branch in a potentially multicultural city. This analysis would suggest that the audience will success by openning a new coffee shop regardless there is already competition around.


## Background

Let's say I want to open a new coffee shop in Mexico City which is known to have a wide veriety of culture (you can thing of any other city with this feature, not necessarily a big city). I might fear I won't have success because there are already coffee shops in the location that I like, I'm afraid of the competition.

Now let's assume that clusters of coffee shops in multicultural cities share the fact that there are parks and museums closeby. The following report will tell me whether is reliable to take the risk by openning a new branch only by looking at the type of venues close to the location.

# 2. The Data

## Description of the Data

The data that we are going to work with are data frames that include information about each coffe shop involved such as name, categories, address, latitude, longitude, distance, postal code, city, state, country, neighborhood and id. The information is obtained from the Foursquare API.

First we import the apropriate libraries.

In [45]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Set up the credentials.

In [82]:
CLIENT_ID = 'C1JSXBICKQBK2ZFP4N1TLFJKVTUUUVXZXK04VCA3KDUXBKHD' # your Foursquare ID
CLIENT_SECRET = 'LEPF0AFLCLTSBB4JP0EQMLFTEHIMICN0HJHUFQ45PG3JW0EP' # your Foursquare Secret
ACCESS_TOKEN = 'H3PYWJXU1FMI5WOPV4SOYH1EFOWCMGFN0CTOW5XOUMFFRSJB' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C1JSXBICKQBK2ZFP4N1TLFJKVTUUUVXZXK04VCA3KDUXBKHD
CLIENT_SECRET:LEPF0AFLCLTSBB4JP0EQMLFTEHIMICN0HJHUFQ45PG3JW0EP


Get the latitude and longitude of Paris, New York City and Toronto

In [83]:
paris = '12 Rue du Sentier, 75002 Paris'
nyc =  '235 Bowery, New York, NY 10002'
toronto = '100 Queen St W, Toronto, Canada'

cities = [paris, nyc, toronto]
locations = []

for address in cities:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    locations.append([location.latitude, location.longitude])
    print(address+", coordinates: "+ str(location.latitude)+" "+str(location.longitude))


12 Rue du Sentier, 75002 Paris, coordinates: 48.8690112 2.345834
235 Bowery, New York, NY 10002, coordinates: 40.7223414 -73.99283154963459
100 Queen St W, Toronto, Canada, coordinates: 43.6536032 -79.38400547469666


Create a data frame for each city with essential information.

In [84]:
data_frames = []
search_query = 'Coffee'

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


for latitude, longitude in locations:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius=2000&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, LIMIT)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    
    data_frames.append(dataframe_filtered)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [81]:
data_frames[0].head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Coffee Crêpes,Coffee Shop,24 quai du Louvre,48.858841,2.340802,"[{'label': 'display', 'lat': 48.85884059518823...",1190,75001,FR,Paris,Île-de-France,France,"[24 quai du Louvre, 75001 Paris, France]",NaN,NaN,52b5c225498eeda0297a855a
1,Starbucks Coffee - kiosque,Café,rue Du 8 Mai 1945,48.875763,2.358857,"[{'label': 'display', 'lat': 48.875763, 'lng':...",1214,75010,FR,Paris,Île-de-France,France,"[rue Du 8 Mai 1945, 75010 Paris, France]",NaN,NaN,5d5a6e02b75d3c000930972a
2,Super Wild Coffee,Café,rue Du 8 Mai 1945,48.875763,2.358857,"[{'label': 'display', 'lat': 48.875763, 'lng':...",1214,75010,FR,Paris,Île-de-France,France,"[rue Du 8 Mai 1945, 75010 Paris, France]",NaN,NaN,5bf955c200b068002dd619da
3,Costa Coffee - zone Eurostar,Café,12 rue De Dunkerque,48.880948,2.355314,"[{'label': 'display', 'lat': 48.880948, 'lng':...",1499,75010,FR,Paris,Île-de-France,France,"[12 rue De Dunkerque, 75010 Paris, France]",NaN,NaN,5d5a6df9912a950007a713cb
4,Coffee Parisien,American Restaurant,4 rue Princesse,48.852607,2.334536,"[{'label': 'display', 'lat': 48.85260711376305...",2004,75006,FR,Paris,Île-de-France,France,"[4 rue Princesse, 75006 Paris, France]",NaN,NaN,4b08f601f964a520981323e3


In [78]:
data_frames[1].head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Coffee Project New York,Coffee Shop,239 E 5th St,btwn 2nd Ave & Bowery,40.727153,-73.989527,"[{'label': 'display', 'lat': 40.727153, 'lng':...",603,10003,US,New York,NY,United States,"[239 E 5th St (btwn 2nd Ave & Bowery), New Yor...",NaN,56054685498e21a5e4c9d5b2
1,WFM Coffee Bar,Coffee Shop,95 E Houston St,At Bowery,40.723627,-73.991291,"[{'label': 'display', 'lat': 40.72362664050156...",193,10002,US,New York,NY,United States,"[95 E Houston St (At Bowery), New York, NY 10002]",Lower East Side,4bae332df964a52045913be3
2,Think Coffee,Coffee Shop,1 Bleecker St,at Bowery,40.725266,-73.992341,"[{'label': 'display', 'lat': 40.72526643084157...",328,10012,US,New York,NY,United States,"[1 Bleecker St (at Bowery), New York, NY 10012]",NaN,491ada01f964a52072521fe3
3,Kam Hing Coffee Shop 金興,Coffee Shop,118 Baxter St,btwn Hester and Canal St,40.718017,-73.999210,"[{'label': 'display', 'lat': 40.71801668499095...",722,10013,US,New York,NY,United States,"[118 Baxter St (btwn Hester and Canal St), New...",NaN,4a118d01f964a52023771fe3
4,COFFEE & CREAM by Oddfellows,Ice Cream Shop,55 E Houston St,at Mott St,40.724707,-73.994281,"[{'label': 'display', 'lat': 40.72470732564802...",290,10012,US,New York,NY,United States,"[55 E Houston St (at Mott St), New York, NY 10...",NaN,5abfe6bbc0af5705a1bcacdb


In [79]:
data_frames[2].head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,HotBlack Coffee,Coffee Shop,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",520,M5V 1Z4,CA,Entertainment District,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",59f784dd28122f14f9d5d63d
1,Timothy's World Coffee,Coffee Shop,427 University Avenue,NaN,43.654053,-79.388090,"[{'label': 'display', 'lat': 43.65405317976302...",332,NaN,CA,NaN,Toronto,ON,Canada,"[427 University Avenue, Toronto ON, Canada]",4b44fc77f964a520cc0026e3
2,Timothy's World Coffee,Coffee Shop,"483 Bay St, Bell Trinity Square",Bell Trinity Square,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",137,M5G 2C9,CA,NaN,Toronto,ON,Canada,"[483 Bay St, Bell Trinity Square (Bell Trinity...",4b0aaa8ef964a520272623e3
3,Timothy's World Coffee,Coffee Shop,425 University Ave,Dundas,43.654270,-79.387448,"[{'label': 'display', 'lat': 43.65427, 'lng': ...",287,M5G 1T6,CA,NaN,Toronto,ON,Canada,"[425 University Ave (Dundas), Toronto ON M5G 1...",53e8acc4498ee294fb100183
4,Timothy's World Coffee,Coffee Shop,401 Bay St.,at Richmond St. W,43.652135,-79.381172,"[{'label': 'display', 'lat': 43.65213455850074...",280,M5H 2Y4,CA,NaN,Toronto,ON,Canada,"[401 Bay St. (at Richmond St. W), Toronto ON M...",4baa9f6cf964a520817a3ae3
